In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

import scipy.stats as stats

import ast #библиотека для предобработки
import warnings

In [2]:
#зададим общий стиль визуализаций
plt.style.use('bmh')

# 1 Загрузка, проверка  и предобработка данных

### 1.1 Загрузка данных

Загрузим начальные датасеты, произведем предварительную предобработку данных

3 датасета:

https://www.kaggle.com/datasets/thedevastator/spotify-tracks-genre-dataset содержт сведения о 114 тыс. композициях
https://www.kaggle.com/datasets/cesaregarza/everynoise-database-with-spotify-features содержит сведения о 544 тыс. композициях
https://www.kaggle.com/datasets/tonygordonjr/spotify-dataset-2023 содержит сведения о 436 тыс. композициях



In [3]:
#spotify-tracks-genre-dataset
df1 = pd.read_csv('C:/Users/Asus/Яндекс_практикум/kaggle spotify/spotify/databases/114k_songs.csv')

In [4]:
#everynoise-database-with-spotify-feature
df2 = pd.read_csv('C:/Users/Asus/Яндекс_практикум/kaggle spotify/spotify/databases/544k_songs.csv')

In [5]:
#spotify-dataset-2023
df3 = pd.read_csv('C:/Users/Asus/Яндекс_практикум/kaggle spotify/spotify/spotify dataset 2023/spotify_data_12_20_2023.csv')

C:\Users\Asus\AppData\Local\Temp\ipykernel_3004\2373894122.py:2: DtypeWarning: Columns (25,26,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('C:/Users/Asus/Яндекс_практикум/kaggle spotify/spotify/spotify dataset 2023/spotify_data_12_20_2023.csv')


In [6]:
#display(df1.info())
#display(df2.info())
#display(df3.info())

Датасет df3 содержит лишние для объединения разобранные столбцы с уникальными жанрами и наименованиями исполнителей, а также различные атрибуты, котоыре не понадобятся для дальнейшего анализа. Удалим их

In [7]:
df3 = df3.drop(['album_id', 'album_name', 'album_popularity', 'album_type', 'label',
                'duration_sec', 'total_tracks', 'track_number', 'followers', 'track_href',
                'type', 'uri', 'release_year', 'release_month', 'rn',
                'artist_0','artist_1','artist_2','artist_3','artist_4','artist_id', 'name',
                'artist_popularity', 'artist_genres', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'analysis_url'], axis=1)

In [8]:
display(df1.info())
display(df2.info())
display(df3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544401 entries, 0 to 544400
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Name              544387 non-null  object 
 1   Id                544401 non-null  object 
 2   Artists           544401 non-null  object 
 3   Artists_Id        544401 non-null  object 
 4   Release           544378 non-null  object 
 5   Duration          544401 non-null  int64  
 6   Genre             544401 non-null  object 
 7   Danceability      544401 non-null  float64
 8   Energy            544401 non-null  float64
 9   Key               544401 non-null  int64  
 10  Loudness          544401 non-null  float64
 11  Speechiness       544401 non-null  float64
 12  Acousticness      544401 non-null  float64
 13  Instrumentalness  544401 non-null  float64
 14  Liveness          544401 non-null  float64
 15  Valeance          544401 non-null  float64
 16  Tempo             54

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375141 entries, 0 to 375140
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artists           375141 non-null  object 
 1   release_date      375119 non-null  object 
 2   track_id          375141 non-null  object 
 3   track_name        375139 non-null  object 
 4   genre_0           209863 non-null  object 
 5   acousticness      374364 non-null  float64
 6   danceability      374364 non-null  float64
 7   duration_ms       374364 non-null  float64
 8   energy            374364 non-null  float64
 9   instrumentalness  374364 non-null  float64
 10  key               374364 non-null  float64
 11  liveness          374364 non-null  float64
 12  loudness          374364 non-null  float64
 13  mode              374364 non-null  float64
 14  speechiness       374364 non-null  float64
 15  tempo             374364 non-null  float64
 16  time_signature    37

None

### 1.2 Предварительная предобработка

1) Для объединения датасетов необходимо нормализовтаь порядок столбцов, их наименования и типы данных. 

Предварительный просмотр датасетов выявил, что, например, наименования исполнителей в датафрейме df1 - строки в которых перечислены артисты через ;, а в датафрейме df2  - значения виде строк, представляющих списки. 

2) В целом датасеты содержат аналогичный атрибуты, но в тоже время имеют существенные различия:

- общее в датасетах

1) id композиции
2) популярность композиции
3) основные характеристики композиции

- различия

1) в df1 отсутствует информация о годе выпуска композиции
2) в df2 отсутсвует информация о наименовании альбома композиции, популярности, наличии ненормативной лексики, моде и временной сигнатуре, популярности
3) df3 в целом соответствует df1


3. Добавления пустых столбцов

- В датасет df1 добавим пустой столбец, который возможно будет содержать год выпуска комозиции
- В датасет df2 добавим пустые столбцы, которые возможно будут содержать значения популярности, моды, сигнатуры и наличия ненормативной лексики
- Датасет df3 пока оставим без изменений

In [9]:
# предобработка df1
# удаление столбца в df 1
df1 = df1.drop(['Unnamed: 0', 'album_name'], axis=1)
#добавление пустых столбцов
df1['release'] = np.nan

In [10]:
# предобработка df2
# преобразуем наименвания столбцов в строчные
df2.columns = df2.columns.str.lower()

#преобразуем значения в столбце artists из строк, представляющих списки в строку с разделителем ;, как в df 1
import ast
#преобразуем строковые представления списков в списки
df2['artists'] = df2['artists'].apply(ast.literal_eval)
# преобразуем списки в строки с разделителем ;
df2['artists'] = df2['artists'].apply(lambda x: ';'.join(x))

#переименуем столбец
df2 = df2.rename(columns={'id' : 'track_id'})

#удалим лишний столбец
df2 = df2.drop(['artists_id'], axis=1)

#добавление пустых столбцов
df2['popularity'] = np.nan
df2['explicit'] = np.nan
df2['mode'] = np.nan
df2['time_signature'] = np.nan

In [11]:
# предобработка df3
#преобразуем значения в столбце artists из строк, представляющих списки в строку с разделителем ;, как в df 1
#преобразуем строковые представления списков в списки
df3['artists'] = df3['artists'].apply(ast.literal_eval)
# преобразуем списки в строки с разделителем ;
df3['artists'] = df3['artists'].apply(lambda x: ';'.join(x))

4) Перестановка столбцов и их переименовывание.

За основу наименований столбцов возьмем df1.

In [12]:
df2 = df2.rename(columns={'name' : 'track_name', 'duration': 'duration_ms', 'genre' : 'track_genre', 'valeance' : 'valence'})
df3 = df3.rename(columns={'track_popularity' : 'popularity', 'release_date' : 'release', 'genre_0':'track_genre'})

In [13]:
#зададим напрямую нужный порядок столбцов
df_col_list = df1.columns.tolist()
df2 = df2[df_col_list]
df3 = df3[df_col_list]

5) Также проверим, какие пересечения есть в наборах данных по track_id

In [14]:
df_list =[df1, df2, df3]
for i in range(len(df_list)):
    for j in range(i + 1, len(df_list)):
        intersection_df = pd.merge(df_list[i], df_list[j], on='track_id')
        print(f"Размер пересечения между df{i+1} и df{j+1}: {intersection_df.shape[0]}")

Размер пересечения между df1 и df2: 21020
Размер пересечения между df1 и df3: 3123
Размер пересечения между df2 и df3: 2917


Т.о. найденные датасеты практически не пересекаются, кроме df1 и df2. Это говорит о том, что возникнут трудности при заполнении пропусков в данных, с другой стороны получим достаточно большой и полный набор объединенных данных.

6) Объединение предобработанных датафреймов

In [15]:
data = pd.concat([df1, df2, df3])

In [16]:
#основная информация
display(data.head())
print('=' * 40)
display(data.info())
print('=' * 40)
print('Количество пропусков:', data.isnull().sum())
print('=' * 40)
print('Количество полных дубликатов :',data.duplicated().sum())
print('=' * 40)

,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,release
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,73.0,230666.0,False,0.676,0.4610,1.0,-6.746,0.0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4.0,acoustic,NaN
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,55.0,149610.0,False,0.420,0.1660,1.0,-17.235,1.0,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4.0,acoustic,NaN
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,57.0,210826.0,False,0.438,0.3590,0.0,-9.734,1.0,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4.0,acoustic,NaN
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,71.0,201933.0,False,0.266,0.0596,0.0,-18.515,1.0,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3.0,acoustic,NaN
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,82.0,198853.0,False,0.618,0.4430,2.0,-9.681,1.0,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4.0,acoustic,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033542 entries, 0 to 375140
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   track_id          1033542 non-null  object 
 1   artists           1033541 non-null  object 
 2   track_name        1033525 non-null  object 
 3   popularity        489131 non-null   float64
 4   duration_ms       1032765 non-null  float64
 5   explicit          489131 non-null   object 
 6   danceability      1032765 non-null  float64
 7   energy            1032765 non-null  float64
 8   key               1032765 non-null  float64
 9   loudness          1032765 non-null  float64
 10  mode              488364 non-null   float64
 11  speechiness       1032765 non-null  float64
 12  acousticness      1032765 non-null  float64
 13  instrumentalness  1032765 non-null  float64
 14  liveness          1032765 non-null  float64
 15  valence           1032765 non-null  float64
 16  t

None

Количество пропусков: track_id                 0
artists                  1
track_name              17
popularity          544411
duration_ms            777
explicit            544411
danceability           777
energy                 777
key                    777
loudness               777
mode                545178
speechiness            777
acousticness           777
instrumentalness       777
liveness               777
valence                777
tempo                  777
time_signature      545178
track_genre         165278
release             114045
dtype: int64
Количество полных дубликатов : 450


In [17]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
popularity,489131.0,15.625699,19.941654,0.0,0.0000,5.000000,27.000,100.000
duration_ms,1032765.0,238574.561356,123806.397274,0.0,177035.0000,218280.000000,272575.000,5828731.000
danceability,1032765.0,0.554021,0.190264,0.0,0.4270,0.572000,0.699,0.999
energy,1032765.0,0.583233,0.268411,0.0,0.3880,0.613000,0.809,1.000
key,1032765.0,5.271209,3.560383,0.0,2.0000,5.000000,8.000,11.000
loudness,1032765.0,-9.955706,6.011136,-60.0,-12.2170,-8.367000,-5.903,5.968
mode,488364.0,0.637881,0.480614,0.0,0.0000,1.000000,1.000,1.000
speechiness,1032765.0,0.097690,0.129986,0.0,0.0364,0.049800,0.094,0.970
acousticness,1032765.0,0.356075,0.355446,0.0,0.0207,0.218000,0.692,0.996
instrumentalness,1032765.0,0.240348,0.363744,0.0,0.0000,0.000694,0.566,1.000


In [18]:
data['track_genre'].nunique()

6015

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033542 entries, 0 to 375140
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   track_id          1033542 non-null  object 
 1   artists           1033541 non-null  object 
 2   track_name        1033525 non-null  object 
 3   popularity        489131 non-null   float64
 4   duration_ms       1032765 non-null  float64
 5   explicit          489131 non-null   object 
 6   danceability      1032765 non-null  float64
 7   energy            1032765 non-null  float64
 8   key               1032765 non-null  float64
 9   loudness          1032765 non-null  float64
 10  mode              488364 non-null   float64
 11  speechiness       1032765 non-null  float64
 12  acousticness      1032765 non-null  float64
 13  instrumentalness  1032765 non-null  float64
 14  liveness          1032765 non-null  float64
 15  valence           1032765 non-null  float64
 16  t

Новые датасеты

In [20]:
df4 = pd.read_csv('C:/Users/Asus/Яндекс_практикум/kaggle spotify/spotify/databases/1m_songs.csv')

In [21]:
df5 = pd.read_csv('C:/Users/Asus/Яндекс_практикум/kaggle spotify/spotify/databases/1_2m_songs.csv')

In [22]:
df5 = df5.rename(columns={'id' : 'track_id'})

In [24]:
df_list =[df1, df2, df3, df4, df5]
for i in range(len(df_list)):
    for j in range(i + 1, len(df_list)):
        intersection_df = pd.merge(df_list[i], df_list[j], on='track_id')
        print(f"Размер пересечения между df{i+1} и df{j+1}: {intersection_df.shape[0]}")

Размер пересечения между df1 и df2: 21020
Размер пересечения между df1 и df3: 3123
Размер пересечения между df1 и df4: 44628
Размер пересечения между df1 и df5: 6041
Размер пересечения между df2 и df3: 2917
Размер пересечения между df2 и df4: 65539
Размер пересечения между df2 и df5: 19198
Размер пересечения между df3 и df4: 21924
Размер пересечения между df3 и df5: 13798
Размер пересечения между df4 и df5: 79909


## 1.3 Предобработка

На стадии предобработки необходимо:
1) удалить лишний столбец, а также дубликаты и пропуски
2) дополнительно проверить столбец 'artists'

### 1.2.1 Удаление дубликатов и пропусков

In [25]:
#убедимся, что действительно есть полные дубликаты
#display(df.sort_values(by='popularity', ascending=False).head(3))
#display(df.query('track_id == "2zg3iJW4fK7KZgHOvJU67z"'))

In [26]:
#рассмотрим пропуск отдельно
#df.query('artists.isna()')

Действительно, в наборе данных есть дубликаты, которые необходимо удалить.

В датасете есть один трек, для которого отсутсвуют атрибуты наименований - артист, песня, альбом. Жанр данной композиции - k-pop. Скорее всего при формировании датасета данные наименования не распознались из-за того, что были записаны корейскими иероглифами. С другой стороны в датасете есть наименования записанные иероглифами, и это не признак жанра.

Рассмотрим пару примеров, из которых можно заключить, что отсутсвие наименования скорее всего исключение из практики. Заменять значения не имеет смысла, поэтому удалим данную строку.

In [27]:
# примеры треков в стиле k-pop
#df.query('track_genre == "k-pop"')[['artists', 'track_name', 'album_name']]

In [28]:
#пример трека с наименованиями в виде иероглифов
#df.query('track_id == "481beimUiUnMUzSbOAFcUT"')

In [29]:
# сохраним в переменную количество строк до удаления
#a = df.shape[0]

In [30]:
# удалим дубликаты
#df = df.drop_duplicates()

In [31]:
# удалим пропуски
#df = df.dropna()

In [32]:
#df.info()

In [33]:
# сохраним в переменную кол-во строк после удаления
#b = df.shape[0]

In [34]:
#print('Удалено {:.4%} данных из датасета'.format(1 - b/a))

### 1.2.2 Столбец 'artists'

Из примеров выше видно, что ряд наименований авторов песен 'artists' иногда содержит 2 и более имени/псевдонима, которые разделены разделителем ";". Выделим уникальные наименования исполнителей для каждой песни
При этом достоверно не известно, по каким правилам назначается порядок следования имен в списке перечисленных исполнителей. Для дальнейшего анализа нам будет важно это понимать.
По информации в открытых источниках порядок может быть:
- алфавитный
- по степени авторства/со-авторства
- по степени участия
- по уровеню вовлеченности в создание
- в зависимости от регалий
- от договоренностей
- случайность

На данном этапе предположим, что порядок зависит от степени авторства и вклада в создание музыкальной композиции.

Стоит учесть, что отстутсвует информация о количестве разделителей в наименовании исполнителя, поэтому выполним процедуру разделения отдельно.

In [35]:
#разделение по разделителю на неизвестное число столбцов
#split_artists = df['artists'].str.split(';', expand = True)

In [36]:
#подсчет исполнителей на 1 песню
#split_artists['artists_counter'] = split_artists.count(axis=1)

In [37]:
#объединение с track_id
#split_artists = pd.concat([df['track_id'], split_artists], axis=1)

In [38]:
#split_artists.sort_values(by='artists_counter', ascending=False).head(10)

Т.о. в наборе существует 5 песен, количество исполнителей в которых более 30, и некоторе количество песен, где исполнителей больше 2-3. Как видно из таблицы выше наименования исполнителей расположены не в алфавитном порядке. Поэтому остановимся на предположении, что в перечне исполнителей 1-м стоит основной исполнитель (возможно, он же и автор).

20 и более участников - это скорее всего совместные записи популярных песен. Так и есть композиция с 38-участниками - кавер на песню 1970 года Мигеля Риоса «Himno a la Alegría»

In [39]:
#df.query('track_id == "7F5rc8a0xD6DCloB2kROq9"')[['artists', 'track_name']]

По гистограмме ниже видно, что в целом для большинства песен характерно не более 4-х исполнителей.

In [40]:
#plt.figure(figsize=(10,5))

#ax1 = plt.subplot(1, 2, 1)
#split_artists['artists_counter'].hist(bins=38)
#plt.xlabel('Кол-во исполнителей в песне')
#plt.ylabel('Кол-во композиций')

#ax1 = plt.subplot(1, 2, 2)
#split_artists['artists_counter'].hist(bins=38)
#plt.xlabel('Кол-во исполнителей в песне')
#plt.ylim(0,10000)
#plt.xlim(0,10)
#plt.show()

In [41]:
#split_artists.groupby('artists_counter')['track_id'].count().reset_index().head(6)

NameError: name 'split_artists' is not defined

С другой стороны, максимум 2 исполнителя примерно в более чем 92% песен. Поэтому добавим в наш начальный датасет дополнительные столбцы с первыми 3-мя исполнителями 

In [ ]:
#df1 = df.copy(deep=True)

In [ ]:
#df1[['artist_1', 'artist_2', 'artist_3']] = df1['artists'].str.split(';', 2 , expand= True )

In [ ]:
#df1.groupby('artist_1')['track_id'].count().reset_index().sort_values(by='track_id', ascending=False)

In [ ]:
#df.pivot_table(index='track_genre', values=['danceability', 'energy', 'explicit'], aggfunc = {'mean'})

In [ ]:
#sns.scatterplot(data=df.query('(track_genre == "edm") or (track_genre == "electro") '), x='energy', y='danceability', hue = 'track_genre')

In [ ]:
#df.groupby('track_genre')['explicit'].mean().reset_index().sort_values(by='explicit', ascending=False).head(20)

# 2 EDA

In [ ]:
#df.describe().T

In [ ]:
#heatm = data.drop(['track_id', 'artists', 'track_name', 'track_genre'], axis=1)

In [ ]:
#heatm.corr().style.background_gradient(cmap='Reds', axis=0)

In [ ]:
#plt.figure(figsize=(10,6))
#sns.heatmap(heatm.corr())
#plt.show()

In [ ]:
#heatm.hist(bins=100, figsize=(20,20))
#plt.show()

In [ ]:
#df['track_genre'].unique()

In [ ]:
#allsongs = pd.read_csv('C:/Users/Asus/Яндекс_практикум/kaggle spotify/spotify/databases/544k_songs.csv')

In [ ]:
#allsongs.query('Genre == "pop"')

In [ ]:
#allsongs.groupby('Genre')['Id'].count().reset_index().sort_values(by='Id', ascending=False)3

In [ ]:
#df.groupby('track_genre')['track_id'].count().reset_index().sort_values(by='track_id', ascending=False)

The columns include: 

- artists (the name of the artist or artists who performed the track), 
- album_name (the title of the album to which the track belongs), 
- track_name (the specific name of each track), 
- popularity (a numerical score indicating the popularity of a song on Spotify ranging from 0 to 100), 
- duration_ms (the duration of each track measured in milliseconds), 
- explicit (a boolean value denoting whether a song contains explicit content or not)
- danceability: Danceability measures how suitable a track is for dancing, ranging from 0 to 1. Tracks with high danceability scores are more energetic and rhythmic, making them ideal for dancing.
- energy:Energy represents intensity and activity within a song on a scale from 0 to 1. Tracks with high energy tend to be more fast-paced and intense.
- oudness: Loudness indicates how loud or quiet an entire song is in decibels (dB). Positive values represent louder songs while negative values suggest quieter ones.
- key: Key refers to different musical keys assigned integers ranging from 0-11,with each number representing a different key. Knowing the key can provide insights into the mood and tone of a song.
- valence: Valence measures the musical positiveness conveyed by a track, ranging from 0 to 1. High valence values indicate more positive or happy tracks, while lower values suggest more negative or sad ones.
- tempo: Tempo is the speed or pace of a song in beats per minute (BPM). It gives an idea about how fast or slow a track is.